Colab 전용 데이터셋 생성 코드

In [ ]:
# 필요 라이브러리 설치
!pip install -q -U langchain-core langchain-community langchain-text-splitters pymupdf

In [ ]:
import os
import time
import json
from google import genai
from google.genai import types
from google.colab import userdata, drive
from tqdm.notebook import tqdm

# PDF 처리용
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# API 키 설정
api_key = userdata.get('vla')
client = genai.Client(api_key=api_key)

# Google Drive 마운트
drive.mount('/content/drive')

In [ ]:
# PDF 경로 지정 (사용자 환경에 맞게 수정)
pdf_filename = "/content/drive/MyDrive/js_mujoco/learn_LLM/src/6g_ai.pdf"

if os.path.exists(pdf_filename):
    print(f"파일 로드 준비 완료: {pdf_filename}")

    # 테스트 로드
    try:
        loader = PyMuPDFLoader(pdf_filename)
        docs = loader.load()
        print(f"문서 로드 성공! 총 {len(docs)} 페이지")
    except Exception as e:
        print(f"문서 로드 중 오류 발생: {e}")
else:
    print(f" 파일을 찾을 수 없습니다: {pdf_filename}")

***함수 정의***

In [ ]:
# 1. PDF 로드 및 분할
def load_and_split_pdf(file_path):
    print(f"Loading PDF with PyMuPDF: {file_path}...")
    loader = PyMuPDFLoader(file_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    chunks = text_splitter.split_documents(docs)
    return chunks

In [ ]:
# 2. 데이터셋 생성 루프
def generate_dataset_gemini(chunks, output_file="dataset.jsonl"):
    final_dataset = []

    base_prompt = """
    당신은 6G 이동통신과 AI 분야의 전문가입니다.
    아래 [Context]를 보고 학습용 질문-답변 쌍 3개를 JSON으로 만드세요.

    [Context] (Page {page_num})
    {context}

    [Format]
    {{ "dataset": [ {{ "instruction": "...", "output": "..." }} ] }}
    """

    print(f" 데이터 생성 시작! (총 {len(chunks)} 청크 처리 예정)")

    for chunk in tqdm(chunks):
        try:
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=base_prompt.format(
                    context=chunk.page_content,
                    page_num=chunk.metadata.get('page', 0) + 1
                ),
                config=types.GenerateContentConfig(
                    temperature=0.5,
                    response_mime_type="application/json"
                )
            )

            data = json.loads(response.text)
            if "dataset" in data:
                for item in data["dataset"]:
                    final_dataset.append({
                        "instruction": item["instruction"],
                        "input": "",
                        "output": item["output"]
                    })
            time.sleep(3) # Rate Limit 방지

        except Exception as e:
            print(f"Error occurred: {e}")
            continue

    with open(output_file, "w", encoding="utf-8") as f:
        for entry in final_dataset:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"\n 저장 완료: {output_file} (총 {len(final_dataset)}개 데이터 쌍)")

In [ ]:
# 3. 데이터 증강 루프
def augment_dataset_gemini(input_file, output_file):
    # 입력 파일 읽기
    with open(input_file, "r", encoding="utf-8") as f:
        original_data = [json.loads(line) for line in f]

    augmented_data = []

    aug_prompt = """
    아래 주어진 질문(Instruction)과 답변(Output)을 보고,
    **답변은 그대로 유지하되 질문의 표현만 다르게 바꾼 유사 질문 3개**를 생성하세요.

    [Original]
    Q: {instruction}
    A: {output}

    [Format]
    {{ "variations": ["유사 질문 1", "유사 질문 2", "유사 질문 3"] }}
    """

    print(f" 데이터 증강 시작! (원본 {len(original_data)}개 -> 목표 약 {len(original_data)*4}개)")

    # 진행바 표시
    for i, entry in enumerate(tqdm(original_data)):
        augmented_data.append(entry) # 원본 데이터도 포함

        try:
            # Gemini 호출
            response = client.models.generate_content(
                model='gemini-2.5-flash', 
                contents=aug_prompt.format(
                    instruction=entry['instruction'],
                    output=entry['output']
                ),
                config=types.GenerateContentConfig(
                    temperature=0.7,
                    response_mime_type="application/json"
                )
            )

            # 응답 파싱
            data = json.loads(response.text)

            if "variations" in data:
                count = 0
                for new_q in data["variations"]:
                    augmented_data.append({
                        "instruction": new_q,
                        "input": "",
                        "output": entry['output']
                    })
                    count += 1
                # (옵션) 잘 되고 있는지 확인용 로그 (너무 많으면 주석 처리)
                # print(f"[{i}] 증강 +{count}개 완료")
            else:
                print(f" [{i}] 응답에 'variations' 키가 없습니다: {data}")

            time.sleep(3) # API 제한 방지

        except Exception as e:
            # [중요] 에러가 나면 이유를 출력해야 함
            print(f" [{i}] 에러 발생: {e}")
            continue

    # 결과 저장
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in augmented_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"\n 증강 완료! 총 {len(augmented_data)}개 데이터가 '{output_file}'에 저장됨.")

# 실행
augment_dataset_gemini("6g_ai_dataset.jsonl", "6g_ai_dataset_augmented.jsonl")

메인

In [ ]:
# 1. 로드 및 분할
chunks = load_and_split_pdf(pdf_filename)

In [ ]:
# 2. 초기 데이터셋 생성
generate_dataset_gemini(chunks, "6g_ai_dataset.jsonl")

In [ ]:
# 3. 데이터 증강
augment_dataset_gemini("6g_ai_dataset.jsonl", "6g_ai_dataset_augmented.jsonl")

In [ ]:
import shutil

# 4. 결과 파일 Google Drive로 저장

# PDF 파일이 있던 폴더 경로를 구합니다 (예: /content/drive/MyDrive/...)
save_dir = os.path.dirname(pdf_filename)

# 저장할 전체 경로 설정
target_file_1 = os.path.join(save_dir, "6g_ai_dataset.jsonl")
target_file_2 = os.path.join(save_dir, "6g_ai_dataset_augmented.jsonl")

print(f"📂 결과 파일을 Google Drive 폴더로 저장합니다.\n경로: {save_dir}")

# 파일 이동 (또는 복사)
if os.path.exists("6g_ai_dataset.jsonl"):
    shutil.copy("6g_ai_dataset.jsonl", target_file_1)
    print(f" 저장 완료: {target_file_1}")
else:
    print(" 원본 데이터셋 파일이 생성되지 않았습니다.")

if os.path.exists("6g_ai_dataset_augmented.jsonl"):
    shutil.copy("6g_ai_dataset_augmented.jsonl", target_file_2)
    print(f" 저장 완료: {target_file_2}")
else:
    print(" 증강 데이터셋 파일이 생성되지 않았습니다.")